# 1 - DEFINIÇÃO BIBLIOTECAS

In [1]:
import pyodbc
import pandas as pd
import json
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email import encoders

# 2 - LEITURA DO JSON DE PARAMETRIZAÇÃO

In [2]:

paramJson 	= "/Notebook/Config/config.json"
file 		= open(paramJson)
dfJson 		= json.load(file)


for tag in dfJson:
    server 		= dfJson[tag]['server']
    database 	= dfJson[tag]['database']
    username 	= dfJson[tag]['username']
    password 	= dfJson[tag]['password']
    fromaddr 	= dfJson[tag]['fromaddr']
    toaddr 		= dfJson[tag]['toaddr']
    chave 		= dfJson[tag]['chave']
    subject 	= dfJson[tag]['subject']
    filename 	= dfJson[tag]['filename']
    file 		= dfJson[tag]['file']
    image 		= dfJson[tag]['image']
    


# 3 - CONEXÃO COM SQL SERVER

In [3]:
cnxn 	= pyodbc.connect('DRIVER={SQL Server};SERVER='+server+\
                      ';DATABASE='+database+\
                      ';UID='+username+\
                      ';PWD='+ password)

cursor 	= cnxn.cursor()


query 	= "SELECT * FROM dbo.TB_Carga;"
df 		= pd.read_sql(query, cnxn)
df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,IdCarga,AbrGrupo,CodRef,IdPacote,NomPacote,DthInicio,TpoExecucao,DthFim
0,1,STG,202301101819,0FB8610F-57B8-44F9-AE25-17DCDD33E6C3,04_PKG_EXEMPL04,2023-01-10 18:19:45.747,I,NaT
1,2,STG,202301101827,0FB8610F-57B8-44F9-AE25-17DCDD33E6C3,04_PKG_EXEMPL04,2023-01-10 18:27:57.410,I,NaT
2,3,STG,202301101829,0FB8610F-57B8-44F9-AE25-17DCDD33E6C3,04_PKG_EXEMPL04,2023-01-10 18:29:13.657,F,2023-01-10 18:29:17.993
3,4,STG,202301101829,0FB8610F-57B8-44F9-AE25-17DCDD33E6C3,04_PKG_EXEMPL04,2023-01-10 18:29:41.483,F,2023-01-10 18:29:45.813
4,5,STG,202301101830,0FB8610F-57B8-44F9-AE25-17DCDD33E6C3,04_PKG_EXEMPL04,2023-01-10 18:30:11.143,I,NaT
5,6,STG,202301101831,0FB8610F-57B8-44F9-AE25-17DCDD33E6C3,04_PKG_EXEMPL04,2023-01-10 18:31:19.440,F,2023-01-10 18:31:30.873


# 4 - ENVIO E-MAIL

In [4]:

df.to_csv(file,sep=';',index=False)

msg 			= MIMEMultipart()
msg['From'] 	= fromaddr 
msg['To']		= toaddr 
msg['Subject'] 	= subject
filename 		= filename

msg.attach(MIMEText('Olá, segue em anexo o relatório com os pacotes que apresentaram falha no processamento. <p><p> <img src="cid:image1">', 'html'))
image = MIMEImage(open(image, 'rb').read())
attachment = open(file, "rb")
p = MIMEBase('application', 'octet-stream')
p.set_payload((attachment).read())
encoders.encode_base64(p)
image.add_header('Content-ID', '<image1>')
msg.attach(image)
p.add_header('Content-Disposition', "attachment; filename= %s" % filename)
msg.attach(p)
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(fromaddr, chave)
text = msg.as_string()
s.sendmail(fromaddr, toaddr, text)
s.quit() 

(221,
 b'2.0.0 closing connection b6-20020a9d4786000000b0066e67925addsm10897759otf.26 - gsmtp')